<font color = Blue>
Research Question 3 (ACS): What are the differences between the 2014 and 2019 5-year American Community Survey datasets? Which variables or labels are missing from the 2014 data but in the 2019 data, and vice versa? Which 2019 variables have changed significantly from the 2014 estimates?

In [1]:
# Import libraries
import os
import pandas as pd
import numpy as np
from functools import reduce

In [2]:
# Read the dictionary csv
acs_dic = pd.read_csv('data/acs5_variable_dict_2014_2019.csv')

# Change label field to upper case
acs_dic['label'] = acs_dic['label'].str.upper()

# Split the labels and store them in a list
acs_dic['label'] = acs_dic['label'].str.split('!!')

In [3]:
# Read other data files for parallel EDA
hill_acs_2014 = pd.read_csv("data/hillsborough_acs5-2014_census.csv")
miami_dade_acs_2014 = pd.read_csv("data/miami_dade_acs5-2014_census.csv")
orange_acs_2014 = pd.read_csv("data/orange_acs5-2014_census.csv")

hill_acs_2019 = pd.read_csv("data/hillsborough_acs5-2019_census.csv")
miami_dade_acs_2019 = pd.read_csv("data/miami_dade_acs5-2019_census.csv")
orange_acs_2019 = pd.read_csv("data/orange_acs5-2019_census.csv")

In [4]:
# Create a df with a list of field names and datasets they're present in
df1 = pd.DataFrame(columns = ['variable_code'], data = hill_acs_2014.columns)
df1['dataset1'] = 'hill_14'
df2 = pd.DataFrame(columns = ['variable_code'], data = miami_dade_acs_2014.columns)
df2['dataset2'] = 'miami_dade_14'
df3 = pd.DataFrame(columns = ['variable_code'], data = orange_acs_2014.columns)
df3['dataset3'] = 'orange_14'
df4 = pd.DataFrame(columns = ['variable_code'], data = hill_acs_2019.columns)
df4['dataset4'] = 'hill_19'
df5 = pd.DataFrame(columns = ['variable_code'], data = miami_dade_acs_2019.columns)
df5['dataset5'] = 'miami_dade_19'
df6 = pd.DataFrame(columns = ['variable_code'], data = orange_acs_2019.columns)
df6['dataset6'] = 'orange_19'

df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['variable_code'], how='outer'), 
                   [df1, df2, df3, df4, df5, df6])

df_merged['datasets'] = df_merged[['dataset1', 'dataset2', 'dataset3', 'dataset4', 'dataset5', 'dataset6',]]\
                        .apply(lambda x: ', '.join(x.dropna()), axis=1)
df_merged = df_merged[['variable_code', 'datasets']]
df_merged = df_merged.loc[df_merged['variable_code'] != 'index']

In [5]:
df_merged.head(5)

,variable_code,datasets
1,DP02_0001E,"hill_14, miami_dade_14, orange_14, hill_19, mi..."
2,DP02_0001PE,"hill_14, miami_dade_14, orange_14, hill_19, mi..."
3,DP02_0002E,"hill_14, miami_dade_14, orange_14, hill_19, mi..."
4,DP02_0002PE,"hill_14, miami_dade_14, orange_14, hill_19, mi..."
5,DP02_0003E,"hill_14, miami_dade_14, orange_14, hill_19, mi..."


In [6]:
# Separate 2014 variable codes from 2019 variable codes
acs_dic_14 = acs_dic.loc[acs_dic['acs_year'] == 2014]
acs_dic_19 = acs_dic.loc[acs_dic['acs_year'] == 2019]

In [7]:
print('Number of variable codes in 2014 dataset:', len(pd.unique(acs_dic_14['variable_code'])))
print('Number of variable codes in 2019 dataset:', len(pd.unique(acs_dic_19['variable_code'])))

Number of variable codes in 2014 dataset: 4088
Number of variable codes in 2019 dataset: 4176


In [8]:
# Format the field names add dataset year as a suffix 
# (except for 'variable_code' field since it'll be used to join the 2 tables)
acs_dic_14.columns = ['{}{}'.format(c, '' if c == 'variable_code' else '_14') for c in acs_dic_14.columns]
acs_dic_19.columns = ['{}{}'.format(c, '' if c == 'variable_code' else '_19') for c in acs_dic_19.columns]

In [9]:
# Merge 2014 dataset, 2019 dataset and df_merged (field names from each file) on 'variable_code'
acs_dic_compare = reduce(lambda  left,right: pd.merge(left,right,on=['variable_code'], how='outer'), 
                   [acs_dic_14, acs_dic_19, df_merged])

# Compare 'concept', 'predicateType', 'predicateOnly' and 'label' fields in both of the datasets
acs_dic_compare['concept_match'] = np.where(acs_dic_compare['concept_14'] == acs_dic_compare['concept_19'], True, False)
acs_dic_compare['predicateType_match'] = np.where(acs_dic_compare['predicateType_14'] == acs_dic_compare['predicateType_19'], True, False)
acs_dic_compare['predicateOnly_match'] = np.where(acs_dic_compare['predicateOnly_14'] == acs_dic_compare['predicateOnly_19'], True, False)
acs_dic_compare['label_match'] = np.where(acs_dic_compare['label_14'] == acs_dic_compare['label_19'], True, False)

## Data dictionary for the new fields:
- *label_14, concept_14, predicateType_14, group_14, limit_14*: labels, concepts, predicateTypes, groups, limits for __2014's__ variable codes

- *label_19, concept_19, predicateType_19, group_19, limit_19*: labels, concepts, predicateTypes, groups, limits for __2019's__ variable codes

<font color = Blue>
    
- *datasets*: variable code is present in these datafiles
    
- *concept_match, predicateType_match, predicateOnly_match, label_match*:(T/F) if 2014's fields match with 2019's fields
    
- *label_same*: common labels in 2014's and 2019's dataset
    
- *label_14_diff*: labels present in 2014's dataset but NOT in 2019's
    
- *label_19_diff*: labels present in 2019's dataset but NOT in 2014's

In [18]:
acs_dic_compare.head(5)

,variable_code,label_14,concept_14,predicateType_14,group_14,limit_14,predicateOnly_14,acs_year_14,label_19,concept_19,...,predicateOnly_19,acs_year_19,datasets,concept_match,predicateType_match,predicateOnly_match,label_match,label_same,label_14_diff,label_19_diff
0,DP02_0019EA,"[ANNOTATION OF ESTIMATE, RELATIONSHIP, POPULAT...",SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,string,DP02,0.0,True,2014.0,"[ANNOTATION OF ESTIMATE, RELATIONSHIP, POPULAT...",SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,...,True,2019.0,NaN,True,True,True,False,"[ANNOTATION OF ESTIMATE, RELATIONSHIP, POPULAT...",[SPOUSE],[HOUSEHOLDER]
1,DP02_0126E,"[ESTIMATE, ANCESTRY, TOTAL POPULATION, DANISH]",SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,int,DP02,0.0,True,2014.0,"[ESTIMATE, ANCESTRY, TOTAL POPULATION, CZECH]",SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,...,True,2019.0,"hill_14, miami_dade_14, orange_14, hill_19, mi...",True,True,True,False,"[ANCESTRY, TOTAL POPULATION, ESTIMATE]",[DANISH],[CZECH]
2,DP02_0072EA,"[ANNOTATION OF ESTIMATE, DISABILITY STATUS OF ...",SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,string,DP02,0.0,True,2014.0,"[ANNOTATION OF ESTIMATE, DISABILITY STATUS OF ...",SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,...,True,2019.0,NaN,True,True,True,False,"[ANNOTATION OF ESTIMATE, DISABILITY STATUS OF ...",[UNDER 18 YEARS],[TOTAL CIVILIAN NONINSTITUTIONALIZED POPULATIO...
3,DP02_0069PMA,"[ANNOTATION OF PERCENT MARGIN OF ERROR, VETERA...",SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,string,DP02,0.0,True,2014.0,"[ANNOTATION OF PERCENT MARGIN OF ERROR, VETERA...",SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,...,True,2019.0,NaN,True,True,True,False,"[ANNOTATION OF PERCENT MARGIN OF ERROR, VETERA...",[CIVILIAN VETERANS],[]
4,DP02_0126M,"[MARGIN OF ERROR, ANCESTRY, TOTAL POPULATION, ...",SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,int,DP02,0.0,True,2014.0,"[MARGIN OF ERROR, ANCESTRY, TOTAL POPULATION, ...",SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,...,True,2019.0,NaN,True,True,True,False,"[ANCESTRY, MARGIN OF ERROR, TOTAL POPULATION]",[DANISH],[CZECH]


In [10]:
# Number of matches
for c in ['concept_match', 'predicateType_match', 'predicateOnly_match', 'label_match']:
    print('\n')
    print(acs_dic_compare[c].value_counts())



True     4088
False     659
Name: concept_match, dtype: int64


True     3872
False     875
Name: predicateType_match, dtype: int64


True     4088
False     659
Name: predicateOnly_match, dtype: int64


False    4203
True      544
Name: label_match, dtype: int64


## How many fields from data dictionary are not present in any of the files?

In [11]:
print('Number of fields from data dictinary that are not present in any of the files:', acs_dic_compare['datasets'].isna().sum())
acs_dic_compare.loc[acs_dic_compare['datasets'].isna()].head(5)

Number of fields from data dictinary that are not present in any of the files: 3132


,variable_code,label_14,concept_14,predicateType_14,group_14,limit_14,predicateOnly_14,acs_year_14,label_19,concept_19,predicateType_19,group_19,limit_19,predicateOnly_19,acs_year_19,datasets,concept_match,predicateType_match,predicateOnly_match,label_match
0,DP02_0019EA,"[ANNOTATION OF ESTIMATE, RELATIONSHIP, POPULAT...",SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,string,DP02,0.0,True,2014.0,"[ANNOTATION OF ESTIMATE, RELATIONSHIP, POPULAT...",SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,string,DP02,0.0,True,2019.0,NaN,True,True,True,False
2,DP02_0072EA,"[ANNOTATION OF ESTIMATE, DISABILITY STATUS OF ...",SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,string,DP02,0.0,True,2014.0,"[ANNOTATION OF ESTIMATE, DISABILITY STATUS OF ...",SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,string,DP02,0.0,True,2019.0,NaN,True,True,True,False
3,DP02_0069PMA,"[ANNOTATION OF PERCENT MARGIN OF ERROR, VETERA...",SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,string,DP02,0.0,True,2014.0,"[ANNOTATION OF PERCENT MARGIN OF ERROR, VETERA...",SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,string,DP02,0.0,True,2019.0,NaN,True,True,True,False
4,DP02_0126M,"[MARGIN OF ERROR, ANCESTRY, TOTAL POPULATION, ...",SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,int,DP02,0.0,True,2014.0,"[MARGIN OF ERROR, ANCESTRY, TOTAL POPULATION, ...",SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,int,DP02,0.0,True,2019.0,NaN,True,True,True,False
5,DP02_0057PMA,"[ANNOTATION OF PERCENT MARGIN OF ERROR, SCHOOL...",SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,string,DP02,0.0,True,2014.0,"[ANNOTATION OF PERCENT MARGIN OF ERROR, SCHOOL...",SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,string,DP02,0.0,True,2019.0,NaN,True,True,True,False


## How many fields from the files are not present in the data dictionary?

In [12]:
print('Number of fields from data dictinary that are not present in any of the files:', \
      len(acs_dic_compare.loc[(acs_dic_compare['concept_14'].isna()) & (acs_dic_compare['concept_19'].isna())].index))
acs_dic_compare.loc[(acs_dic_compare['concept_14'].isna()) & (acs_dic_compare['concept_19'].isna())].reset_index().head(5)

Number of fields from data dictinary that are not present in any of the files: 571


,index,variable_code,label_14,concept_14,predicateType_14,group_14,limit_14,predicateOnly_14,acs_year_14,label_19,...,predicateType_19,group_19,limit_19,predicateOnly_19,acs_year_19,datasets,concept_match,predicateType_match,predicateOnly_match,label_match
0,4176,S1701_C01_001E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,"hill_14, miami_dade_14, orange_14, hill_19, mi...",False,False,False,False
1,4177,S1701_C01_002E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,"hill_14, miami_dade_14, orange_14, hill_19, mi...",False,False,False,False
2,4178,S1701_C01_003E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,"hill_14, miami_dade_14, orange_14, hill_19, mi...",False,False,False,False
3,4179,S1701_C01_004E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,"hill_14, miami_dade_14, orange_14, hill_19, mi...",False,False,False,False
4,4180,S1701_C01_005E,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,"hill_14, miami_dade_14, orange_14, hill_19, mi...",False,False,False,False


In [13]:
# Replace NaN values in 'label_14' and 'label_19' columns with an empty list
acs_dic_compare['label_14'] = acs_dic_compare['label_14'].apply(lambda d: d if isinstance(d, list) else [])
acs_dic_compare['label_19'] = acs_dic_compare['label_19'].apply(lambda d: d if isinstance(d, list) else [])
acs_dic_compare.head(5)

,variable_code,label_14,concept_14,predicateType_14,group_14,limit_14,predicateOnly_14,acs_year_14,label_19,concept_19,predicateType_19,group_19,limit_19,predicateOnly_19,acs_year_19,datasets,concept_match,predicateType_match,predicateOnly_match,label_match
0,DP02_0019EA,"[ANNOTATION OF ESTIMATE, RELATIONSHIP, POPULAT...",SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,string,DP02,0.0,True,2014.0,"[ANNOTATION OF ESTIMATE, RELATIONSHIP, POPULAT...",SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,string,DP02,0.0,True,2019.0,NaN,True,True,True,False
1,DP02_0126E,"[ESTIMATE, ANCESTRY, TOTAL POPULATION, DANISH]",SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,int,DP02,0.0,True,2014.0,"[ESTIMATE, ANCESTRY, TOTAL POPULATION, CZECH]",SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,int,DP02,0.0,True,2019.0,"hill_14, miami_dade_14, orange_14, hill_19, mi...",True,True,True,False
2,DP02_0072EA,"[ANNOTATION OF ESTIMATE, DISABILITY STATUS OF ...",SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,string,DP02,0.0,True,2014.0,"[ANNOTATION OF ESTIMATE, DISABILITY STATUS OF ...",SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,string,DP02,0.0,True,2019.0,NaN,True,True,True,False
3,DP02_0069PMA,"[ANNOTATION OF PERCENT MARGIN OF ERROR, VETERA...",SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,string,DP02,0.0,True,2014.0,"[ANNOTATION OF PERCENT MARGIN OF ERROR, VETERA...",SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,string,DP02,0.0,True,2019.0,NaN,True,True,True,False
4,DP02_0126M,"[MARGIN OF ERROR, ANCESTRY, TOTAL POPULATION, ...",SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,int,DP02,0.0,True,2014.0,"[MARGIN OF ERROR, ANCESTRY, TOTAL POPULATION, ...",SELECTED SOCIAL CHARACTERISTICS IN THE UNITED ...,int,DP02,0.0,True,2019.0,NaN,True,True,True,False


In [14]:
# Identify the difference between labels from 2014 and 2019 dataset

# Common labels for each row:
acs_dic_compare['label_same'] = acs_dic_compare.apply(lambda row: list(set(row['label_14']).intersection(set(row['label_19']))), axis = 1)

# Labels present in 2014's dataset but not in 2019's
acs_dic_compare['label_14_diff'] = acs_dic_compare.apply(lambda row: list(set(row['label_14']).difference(set(row['label_same']))), axis = 1)

# Labels present in 2019's datatset but not in 2014's
acs_dic_compare['label_19_diff'] = acs_dic_compare.apply(lambda row: list(set(row['label_19']).difference(set(row['label_same']))), axis = 1)

In [15]:
# Find number of variable codes associated with each label 
acs_dic_14['label_14'] = acs_dic_14.apply(lambda row: str(row['label_14']), axis = 1)
acs_dic_14_labels = acs_dic_14.groupby('label_14')['variable_code'].apply(list).reset_index()

<ipython-input-15-082515667985>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acs_dic_14['label_14'] = acs_dic_14.apply(lambda row: str(row['label_14']), axis = 1)


In [16]:
# Are there labels with more than 1 variable codes?
acs_dic_14_labels['ct_variable_code'] = acs_dic_14_labels['variable_code'].apply(lambda x: len(x))
acs_dic_14_labels.loc[acs_dic_14_labels['ct_variable_code']>1]

,label_14,variable_code,ct_variable_code
420,"['ANNOTATION OF ESTIMATE', 'SELECTED MONTHLY O...","[DP04_0114EA, DP04_0123EA]",2
425,"['ANNOTATION OF ESTIMATE', 'SEX AND AGE', '18 ...","[DP05_0022EA, DP05_0018EA]",2
438,"['ANNOTATION OF ESTIMATE', 'SEX AND AGE', '65 ...","[DP05_0021EA, DP05_0025EA]",2
928,"['ANNOTATION OF MARGIN OF ERROR', 'SELECTED MO...","[DP04_0123MA, DP04_0114MA]",2
933,"['ANNOTATION OF MARGIN OF ERROR', 'SEX AND AGE...","[DP05_0022MA, DP05_0018MA]",2
946,"['ANNOTATION OF MARGIN OF ERROR', 'SEX AND AGE...","[DP05_0021MA, DP05_0025MA]",2
1436,"['ANNOTATION OF PERCENT MARGIN OF ERROR', 'SEL...","[DP04_0123PMA, DP04_0114PMA]",2
1441,"['ANNOTATION OF PERCENT MARGIN OF ERROR', 'SEX...","[DP05_0018PMA, DP05_0022PMA]",2
1454,"['ANNOTATION OF PERCENT MARGIN OF ERROR', 'SEX...","[DP05_0021PMA, DP05_0025PMA]",2
1944,"['ANNOTATION OF PERCENT', 'SELECTED MONTHLY OW...","[DP04_0114PEA, DP04_0123PEA]",2


In [17]:
acs_dic_14[acs_dic_14['variable_code'].isin(['DP04_0114EA', 'DP04_0123EA'])]

,variable_code,label_14,concept_14,predicateType_14,group_14,limit_14,predicateOnly_14,acs_year_14
2844,DP04_0114EA,"['ANNOTATION OF ESTIMATE', 'SELECTED MONTHLY O...",SELECTED HOUSING CHARACTERISTICS,string,DP04,0,True,2014
3433,DP04_0123EA,"['ANNOTATION OF ESTIMATE', 'SELECTED MONTHLY O...",SELECTED HOUSING CHARACTERISTICS,string,DP04,0,True,2014
